In [1]:
import os
os.environ['KAGGLE_USERNAME'] = '#######' 
os.environ['KAGGLE_KEY'] = '######'
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model

In [3]:
train = pd.read_csv('/content/train.csv.zip')
test = pd.read_csv('/content/train.csv.zip')

In [4]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
train.shape

(159571, 8)

In [8]:
X = train.comment_text.values
y = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((143613,), (15958,), (143613, 6), (15958, 6))

In [11]:
# parameters
vocab_size = 20000
embedding_dim = 128
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [12]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [13]:
training_sequences = tokenizer.texts_to_sequences(X_train)
train_pad = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
test_pad = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [35]:
input = Input(shape = (max_length,))
embed_size = 128
x = Embedding(vocab_size, embed_size)(input)
x = LSTM(60, return_sequences = True, name = "lstm_layer")(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation = "relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation = "sigmoid")(x)
model = Model(inputs = input, outputs = x)
model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

In [36]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 50, 128)           2560000   
                                                                 
 lstm_layer (LSTM)           (None, 50, 60)            45360     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 60)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_4 (Dropout)         (None, 60)                0         
                                                                 
 dense_4 (Dense)             (None, 50)                3050      
                                                           

In [37]:
batch_size = 32
epochs = 2

history = model.fit(train_pad, y_train, batch_size = batch_size, epochs = epochs, validation_data=(test_pad, y_test), verbose=2)

Epoch 1/2
4488/4488 - 237s - loss: 0.0691 - accuracy: 0.9547 - val_loss: 0.0515 - val_accuracy: 0.9935 - 237s/epoch - 53ms/step
Epoch 2/2
4488/4488 - 235s - loss: 0.0490 - accuracy: 0.9786 - val_loss: 0.0489 - val_accuracy: 0.9935 - 235s/epoch - 52ms/step
